# Project 3 : NumPy, matplotlib, and Python functions

## Instructions

### Description

Below is a guided exercise in doing linear regression using a gradient descent algorithm.  The goal here isn't to teach you the gradient descent algorithm, or scientific computing in general, but to give you experience with using NumPy for matrix-vector math, matplotlib for simple visualizations, and more Python (specifically, writing functions).

### Grading

For grading purposes, we will clear all outputs from all your cells and then run them all from the top.  Please test your notebook in the same fashion before turning it in.

### Submitting Your Solution

To submit your notebook, first clear all the cells (this won't matter too much this time, but for larger data sets in the future, it will make the file smaller).  Then use the File->Download As->Notebook to obtain the notebook file.  Finally, submit the notebook file on Canvas.


## Setup Code

Feel free to look through this and see what it is doing, but basically it is just setting up a synthetic regression problem for you, very much like we did in class.  Just make sure you execute the cell before going on to the problems!  If you wish to see the training and testing points, uncomment the last two lines in the cell.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

def f(X):
    return 1/(1 + np.exp(-X))

n = 50
np.random.seed(12345)
trainX = np.random.random(n) * 10 - 5
trainY = f(trainX) + np.random.randn(n) * 0.125
testX = np.random.random(n) * 10 - 5
testY = f(testX) + np.random.randn(n) * 0.125
#plt.plot(trainX, trainY, 'b.', testX, testY, 'gx')
#plt.show()

#### Problem 1: Features (10 points)
For this problem, you need to create a Python function named `phi` that will produce a set of between 5 and 8 features for a given X input, such as the trainX vector created above.  This is basically what we did in lectures 5-7 to generate our design matrix $\Phi$.  The difference is that in our lectures we used powers of $x$ exclusively, where as below I would like you to use at least two non-polynomial functions of $x$.  You can use any functions you want; consider trigonometric functions, logarithmic and exponential functions, radial basis function (Gaussians), etc.  The only functions you are forbidden to use are functions generating a sigmoid such as the one generating the synthetic data for this problem (this includes the arctangent function)!  It is recommended, but not required, that one of your features is an intercept term (all 1's).

Note the triple-quoted string at the top of the function - this produces a documentation comment for the function (visible when you do help(function) or function?).  You should get in the habit of making these for your code.

Note: We discussed a variation of this in the `06-ml-beginnings lecture` ... your code will just have to create a list containing different functions of x to send into the np.array() instead of using all powers of x like we did for trainX in the example: `Phi = np.array([trainX ** p for p in range(6)]).T`

In [ ]:
def phi(X):
    '''
    Return a design matrix (a NumPy ndarray) whose columns are functions of the input vector, X.
    '''
    Phi = np.array([X**0, X**1, X**2, X**3, np.cos(X), np.exp(X), np.sin(X)]).T
    
    return Phi

You should probably check the output of phi(trainX) below, just to make sure none of your functions generated NaNs (Not a Number values) or infs (infinities) - these can result, e.g, from taking the log of a negative number or division by a number close to zero.

In [ ]:
#calls your newly created phi function above for testing
Phi = phi(trainX)

#### Problem 2: Gradient Descent (20 points)
The basic idea behind [gradient descent](https://en.wikipedia.org/wiki/Gradient_descent) is that you can minimize a parametrized function by simply taking small steps in the negative direction of the derivative of the function.  I'm not going to go into too many details here, since [Wikipedia's article](https://en.wikipedia.org/wiki/Gradient_descent) does such a good job.  Also see [Andrew Ng's class notes](https://datajobs.com/data-science-repo/Generalized-Linear-Models-[Andrew-Ng].pdf) on the topic for a more thorough mathematical treatment.

In our cases, we are interesting in minimizing the square error in our linear approximation with respect to our training data.  That is, we want to minimize
$$ \begin{align}
     & \sum_{i=0}^n (\hat{f}(x_i) - y_i)^2 \\
   = & \sum_{i=0}^n (\phi_0(x_i) w_0 + \phi_1(x_i) w_1 + \dots + \phi_k(x_i) w_k - y_i)^2 \\ 
   = & \|\Phi w - \textbf{y}\|^2 
   \end{align}
$$

Since all of our training data (the $x_i$ and $y_i$ terms) are known, the only variables we can optimize on are the $w$ variables. The gradient is (skipping over a *lot* of linear algebra):

$$ \begin{align}
       & \nabla \|\Phi w - \textbf{y}\|^2  \\
     = & \Phi^T (\Phi w - \textbf{y})
   \end{align}
$$

The gradient descent algorithm asks us, then, to start with some guess for $\textbf{w}$ and repeatedly subtract some small multiple of the gradient from it, until $\textbf{w}$ converges:

$$ \textbf{w} = \textbf{w} + \alpha \Phi^T (\textbf{y} - \Phi \textbf{w}) $$

Convergence is a somewhat tricky issue, and the step size $\alpha$ is a key to getting good convergence; a large $\alpha$ speeds up the algorithm, but can miss the optimum by stepping too far (and can even diverge); a small $\alpha$ may result in a better optimum, but at the cost of speed.  You may have to play around a bit with different values for $\alpha$ until you get a good result; print out the norm of the change in `w` at each step to see if you are converging or diverging, or put in an `if` statement to kill the function when the norm starts growing large.  In testing this assignment, I found I had to use a pretty small $\alpha$ to prevent divergence; 0.00001 worked for me.  However, you will get different results depending on your features.

So, you will write a function `lr` which takes in $\Phi$ (generated by your `phi` function above applied to `trainX`) and $y$ (`trainY`) and returns a vector `w` computed by gradient descent.  Follow the steps below to compute w:

 1. Start with `w` as the zero vector.
 2. Initialize `norm`, `delta`, and step counter.
 3. Calculate `delta` (The amount `w` is going to change calculated as: $\alpha \Phi^T (\textbf{y} - \Phi \textbf{w})$ )
 4. Update `w` and step counter
 5. Calculate `norm` (the norm of a vector is the square root of the dot product of the vector with itself)
 6. Repeat steps 3-5 until counter is greateer than 100,000 or `norm` is less than 1e-6. 

Before returning, your function should print out the number of iterations (gradient descent steps) taken.

In [ ]:
def lr(Phi, y):
    '''
    Computes a linear regression result w minimizing ||Phi w - y||^2 via gradient descent.
    '''    
    # Initializing variables
    w = np.array([0] * Phi.shape[1])
    norm = 1
    delta = 0.0001*Phi.T@(y-Phi@w)
    counter = 0
    lastNorm = 2
    
    # Gradient descent loop
    while norm >= 1e-6 and counter <= 100000:
        counter += 1 
        delta = 0.00001*Phi.T@(y-Phi@w)
        w = w + delta
        norm = np.sqrt(delta.T @ delta)
        if norm - lastNorm > 0.01:
            print(f"Steps taken: {counter}")
            break
        lastNorm = norm
        
    print(f"Steps taken: {counter}")
    return w

#### Problem 3: Compute RMSE (10 points)
Use your `lr` function above to compute a solution, then using your solution ($\hat f(x)$), compute the RMSE of your result on **both the training and the test data sets** , and print them below. (For reference, I was able to achieve 0.11574 with 8 features.) Remeber that MSE is:

 $$ MSE = \frac{1}{n} \sum_{i = 1}^n (y_i - \hat f(x_i))^2 $$



In [ ]:
# Call the linear regression
w = lr(Phi, trainY)

# Calculate the RMSE
def rmse(Phi, w, y):
    diff = y - (Phi @ w)
    RMSE = np.sqrt(sum(diff**2)/n)
    return RMSE

# Print the results
print("RMSE Training: ", rmse(Phi, w, trainY))
print("RMSE Test: ", rmse(Phi, w, testY))


#### Problem 4: Plot Results (10 points)
You are going to create three plots below. On the first plot, show the training and testing points (using markers of different colors), and a line plot of your learned function (compute $\hat{y}$ for $x \in (-5,5)$ at increments of 0.1).  This can be accomplished using `plt.plot(...)`, following the examples in the lectures.  Don't forget you can also look at the matplotlib documentation - there's a link under the Help menu in your notebook.  If you feel especially daring, you can try doing the plot in [bokeh](http://bokeh.pydata.org/) instead.

Your second and third plots should be of the residuals on the training set.  (The residuals are the errors, or differences between the training $y$ values and the corresponding $\hat{y}$ values.)  There are various ways to look at the residuals; for the second plot, use `plt.scatter(...)` (or bokeh equivalent) to plot the training $x$ values versus the residuals for the training $y$ values.  If the values are roughly equally distributed above and below the $x$ axis, then our approximation was probably pretty good.

The third plot should use `plt.hist(...)` (or bokeh equivalent) to show a histogram of the residuals (use 5 bins or so).  Given the small number of samples, it is unlikely you'll see a very normal distribution, but it should be vaguely normal, assuming we nearly approximated $f(x)$.

In [ ]:
# Calculate the Predicted Values and create a Function for Later
def f(x,w):
    return w[0] + w[1]*x + w[2]*x**2 + w[3]*x**3 + w[4]*np.cos(x) + w[5]*np.exp(x) + w[6]*np.sin(x)
X = np.arange(-5,5,0.1)
phi_hat = phi(X)
y_hat = phi_hat @ w

# Line Plot
labels = ["Training Data", "Test Data", "Linear Regression"]
plt.plot(trainX, trainY, 'b.', testX, testY, 'gx', X, y_hat, 'r-')
plt.legend(labels)
plt.title("Linear Regression of Training and Test Data")
plt.ylabel("Y Value")
plt.xlabel("X Value")
plt.show()

# Residual Plot for Training Data
residuals = trainY-(phi(trainX)@w)    
plt.scatter(trainX, residuals, label="Residuals")
plt.axhline(y=0, color = 'k', linestyle = ":")
plt.legend()
plt.title("Residuals Plot: Training Values minus Predicted Values")
plt.ylabel("Residuals")
plt.xlabel("X Value")
plt.show()

# Histogram of Residuals
plt.hist(residuals, bins = 7)
plt.title("Histogram of Residuals")
plt.ylabel("Frequency")
plt.xlabel("Residual")
plt.show()